# Fine-tuning com LoRA

### Autenticação

In [ ]:
from scripts.authentcation import authenticate_huggingface

authenticate_huggingface()

### Preparação dos dados

In [ ]:
from datasets import load_dataset
from scripts.data import generate_training_messages

training_dataset = load_dataset('marmal88/skin_cancer', split='train')
validation_dataset = load_dataset('marmal88/skin_cancer', split='validation')

training_dataset_size = int(input('Enter the size of the training data: '))
validation_data_size = training_dataset_size * 0.1

training_messages = generate_training_messages(training_dataset, training_dataset_size)
validation_messages = generate_training_messages(validation_dataset, validation_data_size)

### Inicialização do LLaMa 3.2

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    'unsloth/Llama-3.2-11B-Vision-Instruct',
    load_in_4bit = False,
    use_gradient_checkpointing = 'unsloth',
)

### Configuração de treinamento

In [ ]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,

    r=64,
    lora_alpha=64,
    lora_dropout=0.05,
    bias='none',
    random_state=3407,
    use_rslora=True,
    loftq_config=None
)

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    train_dataset=training_messages,
    eval_dataset=validation_messages,
    args=SFTConfig(
        eval_strategy='steps',
        learning_rate=1e-4,
        weight_decay=0.01,
        lr_scheduler_type='constant',
        seed=3407,
        bf16=is_bf16_supported(),
        fp16=not is_bf16_supported(),
        remove_unused_columns=False,
        optim='paged_adamw_32bit',
        report_to='tensorboard',
        logging_steps=0.05,
        output_dir='outputs',
        dataset_text_field='',
        dataset_kwargs={'skip_prepare_dataset': True},
        dataset_num_proc=4,
        max_seq_length=2048
    ),
)

### Treinamento

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")

### Salvamento

In [ ]:
peft_model = trainer.model

model.save_pretrained(f'../weights/LLaMA_DERM_QLoRA_{training_dataset_size}_11B')
tokenizer.save_pretrained(f'../weights/LLaMA_DERM_QLoRA_{training_dataset_size}_11B')